# Resources : https://medium.com/@vipra_singh

# Finetuning Bert Basics

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the pre-trained BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Encoding / Tokenization

1. **input_ids:** provide the actual content of the input text.
2. **attention_mask:** ensures that the model focuses on the actual content and not the padding.
3. **token_type_ids:**  help the model distinguish between different segments of the input.

In [ ]:
# Tokenize a single text for clarity
text = ["I love programming."]
encoding = tokenizer(text, truncation=True, padding='max_length', max_length=10, return_tensors='tf')

# Decode the token IDs back to tokens
tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'][0].numpy())

print("Text:", text)
print("Tokens:", tokens)
print("Token IDs:", encoding['input_ids'][0].numpy())
print("Token Type IDs:", encoding['token_type_ids'][0].numpy())
print("Attention Mask:", encoding['attention_mask'][0].numpy())
print("encoding:",encoding['input_ids'][0].numpy().shape)

Text: ['I love programming.']
Tokens: ['[CLS]', 'i', 'love', 'programming', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Token IDs: [ 101 1045 2293 4730 1012  102    0    0    0    0]
Token Type IDs: [0 0 0 0 0 0 0 0 0 0]
Attention Mask: [1 1 1 1 1 1 0 0 0 0]
encoding: (10,)


In [ ]:
# Example dataset
texts = ["I love programming.", "The weather is nice today.", "I'm feeling great!"]
labels = [1, 0, 1]  # Binary classification labels (e.g., 1 for positive, 0 for negative)

# Tokenize the texts
encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors='tf')

# Convert labels to tensor
labels = tf.convert_to_tensor(labels)

# Create a dataset
dataset = tf.data.Dataset.from_tensor_slices((dict(encodings), labels))
dataset = dataset.shuffle(len(texts)).batch(2)


In [ ]:
from transformers import TFBertForSequenceClassification

# Load the pre-trained BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Fine-tune the model
model.fit(dataset, epochs=3)


Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2/2 [==============================] - 117s 1s/step - loss: 0.7091 - accuracy: 0.3333
Epoch 2/3
2/2 [==============================] - 3s 1s/step - loss: 0.6852 - accuracy: 0.3333
Epoch 3/3
2/2 [==============================] - 4s 2s/step - loss: 0.5177 - accuracy: 1.0000


In [ ]:
# Example evaluation texts and labels
eval_texts = ["I hate bugs.", "Today is a good day."]
eval_labels = [0, 1]

# Tokenize the evaluation texts
eval_encodings = tokenizer(eval_texts, truncation=True, padding=True, max_length=128, return_tensors='tf')

# Convert labels to tensor
eval_labels = tf.convert_to_tensor(eval_labels)

# Create an evaluation dataset
eval_dataset = tf.data.Dataset.from_tensor_slices((dict(eval_encodings), eval_labels))
eval_dataset = eval_dataset.batch(2)

# Evaluate the model
loss, accuracy = model.evaluate(eval_dataset)
print(f'Loss: {loss}, Accuracy: {accuracy}')


1/1 [==============================] - 17s 17s/step - loss: 0.8718 - accuracy: 0.0000e+00
Loss: 0.8718075752258301, Accuracy: 0.0


# Fintuning Model

In [ ]:
import tensorflow as tf

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")


GPU found


In [ ]:
# from transformers import (
#     BertTokenizer, TFBertForSequenceClassification,
#     DistilBertTokenizer, TFDistilBertForSequenceClassification,
#     RobertaTokenizer, TFRobertaForSequenceClassification,
#     XLMRobertaTokenizer, TFXLMRobertaForSequenceClassification,
#     AlbertTokenizer, TFAlbertForSequenceClassification
# )

# # Dictionary mapping model names to their respective tokenizers and model classes
# model_tokenizer_mapping = {
#     'bert-base-uncased': (BertTokenizer, TFBertForSequenceClassification),
#     'bert-large-uncased': (BertTokenizer, TFBertForSequenceClassification),
#     'bert-base-cased': (BertTokenizer, TFBertForSequenceClassification),
#     'bert-large-cased': (BertTokenizer, TFBertForSequenceClassification),
#     'bert-base-multilingual-uncased': (BertTokenizer, TFBertForSequenceClassification),
#     'bert-base-multilingual-cased': (BertTokenizer, TFBertForSequenceClassification),
#     'bert-base-chinese': (BertTokenizer, TFBertForSequenceClassification),
#     'bert-base-german-cased': (BertTokenizer, TFBertForSequenceClassification),
#     'bert-large-uncased-whole-word-masking': (BertTokenizer, TFBertForSequenceClassification),
#     'bert-large-cased-whole-word-masking': (BertTokenizer, TFBertForSequenceClassification),
#     'bert-large-uncased-whole-word-masking-finetuned-squad': (BertTokenizer, TFBertForSequenceClassification),
#     'bert-large-cased-whole-word-masking-finetuned-squad': (BertTokenizer, TFBertForSequenceClassification),
#     'bert-base-cased-finetuned-mrpc': (BertTokenizer, TFBertForSequenceClassification),
#     'bert-base-german-dbmdz-cased': (BertTokenizer, TFBertForSequenceClassification),
#     'bert-base-german-dbmdz-uncased': (BertTokenizer, TFBertForSequenceClassification),
#     'distilbert-base-uncased': (DistilBertTokenizer, TFDistilBertForSequenceClassification),
#     'distilbert-base-cased': (DistilBertTokenizer, TFDistilBertForSequenceClassification),
#     'roberta-base': (RobertaTokenizer, TFRobertaForSequenceClassification),
#     'roberta-large': (RobertaTokenizer, TFRobertaForSequenceClassification),
#     'xlm-roberta-base': (XLMRobertaTokenizer, TFXLMRobertaForSequenceClassification),
#     'xlm-roberta-large': (XLMRobertaTokenizer, TFXLMRobertaForSequenceClassification),
#     'albert-base-v2': (AlbertTokenizer, TFAlbertForSequenceClassification),
#     'albert-large-v2': (AlbertTokenizer, TFAlbertForSequenceClassification),
#     'albert-xlarge-v2': (AlbertTokenizer, TFAlbertForSequenceClassification),
#     'albert-xxlarge-v2': (AlbertTokenizer, TFAlbertForSequenceClassification),
# }

# # Function to load model and tokenizer using the mapping
# def load_model_and_tokenizer(model_name, num_labels=4):
#     if model_name not in model_tokenizer_mapping:
#         raise ValueError(f"Model name {model_name} is not recognized. Please choose from the following: {list(model_tokenizer_mapping.keys())}")

#     tokenizer_class, model_class = model_tokenizer_mapping[model_name]
#     tokenizer = tokenizer_class.from_pretrained(model_name)
#     model = model_class.from_pretrained(model_name, num_labels=num_labels)
#     return model, tokenizer



In [ ]:

# # Example usage
# model_name = 'distilbert-base-uncased'  # Change this to any other model if needed

# # Load model and tokenizer
# model, tokenizer = load_model_and_tokenizer(model_name)


In [ ]:

# # Prepare the dataset
# train_dataset = prepare_dataset('ag_news', 'train', tokenizer, batch_size=16, max_length=128)
# eval_dataset = prepare_dataset('ag_news', 'test', tokenizer, batch_size=16, max_length=128)


In [ ]:
# # Train the model
# train_model(model, train_dataset, epochs=3)


In [ ]:
# # Evaluate the model
# evaluate_model(model, eval_dataset)

## Finetune-bottle neck layers
 - Note : latest trasformers having some issues so please use !pip install transformers==4.37.2

In [ ]:
!pip install transformers==4.37.2

In [ ]:
# from transformers import BertTokenizer, TFBertModel
# import tensorflow as tf

# # Load pre-trained BERT tokenizer and model
# model_name = 'bert-base-uncased'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# bert_model = TFBertModel.from_pretrained(model_name)


In [ ]:
# bert_model.summary()

In [ ]:
# !pip install transformers==4.37.2

In [ ]:
# from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
# import tensorflow as tf
# from tensorflow.keras.layers import Dense, Input, Dropout
# from tensorflow.keras.models import Model

# # Load pre-trained model and tokenizer
# model_name = 'bert-base-uncased'
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# auto_model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


In [ ]:
# bert_outputs.shape

In [ ]:
# bottleneck_dim=256
# num_labels=2
# input_ids = Input(shape=(128,), dtype=tf.int32, name='input_ids')
# attention_mask = Input(shape=(128,), dtype=tf.int32, name='attention_mask')
# token_type_ids = Input(shape=(128,), dtype=tf.int32, name='token_type_ids')

# bert_outputs = auto_model.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
# cls_output = bert_outputs.pooler_output  # Use pooler_output for [CLS] token

# bottleneck_output = Dense(bottleneck_dim, activation='relu')(cls_output)
# dropout_output = Dropout(0.1)(bottleneck_output)
# logits = Dense(num_labels, activation='softmax')(dropout_output)

# model = Model(inputs=[input_ids, attention_mask, token_type_ids], outputs=logits)
# model.summary()

In [ ]:
# # Create custom model with bottleneck architecture
# def create_custom_model(auto_model, bottleneck_dim=256, num_labels=2):
#     input_ids = Input(shape=(128,), dtype=tf.int32, name='input_ids')
#     attention_mask = Input(shape=(128,), dtype=tf.int32, name='attention_mask')
#     token_type_ids = Input(shape=(128,), dtype=tf.int32, name='token_type_ids')

#     bert_outputs = auto_model.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
#     cls_output = bert_outputs.pooler_output  # Use pooler_output for [CLS] token

#     bottleneck_output = Dense(bottleneck_dim, activation='relu')(cls_output)
#     dropout_output = Dropout(0.1)(bottleneck_output)
#     logits = Dense(num_labels, activation='softmax')(dropout_output)

#     model = Model(inputs=[input_ids, attention_mask, token_type_ids], outputs=logits)
#     return model

# # Create the custom model
# bottleneck_dim = 256
# num_labels = 2
# custom_model = create_custom_model(auto_model, bottleneck_dim, num_labels)

# # Check for GPU availability
# if tf.test.gpu_device_name():
#     print('GPU found')
# else:
#     print("No GPU found")


In [ ]:
# custom_model.summary()

In [ ]:

# # Check for GPU availability
# if tf.test.gpu_device_name():
#     print('GPU found')
# else:
#     print("No GPU found")

# # Dummy dataset
# texts = ["I love programming.", "The weather is nice today.", "I'm feeling great!"]
# labels = [1, 0, 1]

# # Tokenize the texts
# encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors='tf')
# input_ids = encodings['input_ids']
# attention_mask = encodings['attention_mask']
# token_type_ids = encodings['token_type_ids']

# # Create TensorFlow dataset
# dataset = tf.data.Dataset.from_tensor_slices(({
#     'input_ids': input_ids,
#     'attention_mask': attention_mask,
#     'token_type_ids': token_type_ids
# }, labels)).shuffle(len(texts)).batch(2)

# # Compile the model
# custom_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
#                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#                      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# # Train the model
# custom_model.fit(dataset, epochs=3)


In [ ]:
from transformers import AutoTokenizer, TFAutoModel, AutoConfig
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout, Concatenate
from tensorflow.keras.models import Model

# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name, output_hidden_states=True)
bert_model = TFAutoModel.from_pretrained(model_name, config=config)


### Last Hidden State Vector with Bottleneck

In [ ]:
def create_model_1(bert_model, bottleneck_dim=256, num_labels=2):
    input_ids = Input(shape=(128,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(128,), dtype=tf.int32, name='attention_mask')
    token_type_ids = Input(shape=(128,), dtype=tf.int32, name='token_type_ids')

    # Get BERT outputs
    bert_outputs = bert_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    last_hidden_state = bert_outputs.last_hidden_state  # Shape: (batch_size, sequence_length, hidden_size)
    cls_output = last_hidden_state[:, 0, :]  # Extract [CLS] token. Shape: (batch_size, hidden_size)

    print(f"Shape of last_hidden_state: {last_hidden_state.shape}")
    print(f"Shape of cls_output: {cls_output.shape}")

    bottleneck_output = Dense(bottleneck_dim, activation='relu')(cls_output)
    dropout_output = Dropout(0.1)(bottleneck_output)
    logits = Dense(num_labels, activation='softmax')(dropout_output)

    model = Model(inputs=[input_ids, attention_mask, token_type_ids], outputs=logits)
    return model


In [ ]:
# Dummy dataset
texts = ["I love programming.", "The weather is nice today.", "I'm feeling great!"]
labels = [1, 0, 1]

# Tokenize the texts
encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=128, return_tensors='tf')
input_ids = encodings['input_ids']
attention_mask = encodings['attention_mask']
token_type_ids = encodings['token_type_ids']


In [ ]:
# Create TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': input_ids,
    'attention_mask': attention_mask,
    'token_type_ids': token_type_ids
}, labels)).shuffle(len(texts)).batch(2)


In [ ]:
# Create and compile Model 1
model_1 = create_model_1(bert_model)
model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Train Model 1
model_1.fit(dataset, epochs=3)


Shape of last_hidden_state: (None, 128, 768)
Shape of cls_output: (None, 768)
Epoch 1/3


2/2 [==============================] - 47s 226ms/step - loss: 0.6463 - sparse_categorical_accuracy: 0.6667
Epoch 2/3
2/2 [==============================] - 0s 179ms/step - loss: 0.3800 - sparse_categorical_accuracy: 1.0000
Epoch 3/3
2/2 [==============================] - 0s 88ms/step - loss: 0.3756 - sparse_categorical_accuracy: 0.6667


### Enabling the Last Three Layers and Training with Bottleneck

In [ ]:
def create_model_2(bert_model, bottleneck_dim=256, num_labels=2):
    input_ids = Input(shape=(128,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(128,), dtype=tf.int32, name='attention_mask')
    token_type_ids = Input(shape=(128,), dtype=tf.int32, name='token_type_ids')

    # Get BERT outputs
    bert_outputs = bert_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    last_hidden_state = bert_outputs.last_hidden_state  # Shape: (batch_size, sequence_length, hidden_size)
    cls_output = last_hidden_state[:, 0, :]  # Extract [CLS] token. Shape: (batch_size, hidden_size)

    print(f"Shape of last_hidden_state: {last_hidden_state.shape}")
    print(f"Shape of cls_output: {cls_output.shape}")

    bottleneck_output = Dense(bottleneck_dim, activation='relu')(cls_output)
    dropout_output = Dropout(0.1)(bottleneck_output)
    logits = Dense(num_labels, activation='softmax')(dropout_output)

    model = Model(inputs=[input_ids, attention_mask, token_type_ids], outputs=logits)

    # Freeze all layers except the last three
    for layer in bert_model.layers[:-3]:
        layer.trainable = False

    return model


### Using the Last Three Hidden States with Bottleneck

In [ ]:
def create_model_3(bert_model, bottleneck_dim=256, num_labels=2):
    input_ids = Input(shape=(128,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(128,), dtype=tf.int32, name='attention_mask')
    token_type_ids = Input(shape=(128,), dtype=tf.int32, name='token_type_ids')

    # Get BERT outputs
    bert_outputs = bert_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    hidden_states = bert_outputs.hidden_states  # List of hidden states for each layer
    last_hidden_state = hidden_states[-1] # Shape: (batch_size, sequence_length, hidden_size)
    second_last_hidden_state = hidden_states[-2] # Shape: (batch_size, sequence_length, hidden_size)
    third_last_hidden_state = hidden_states[-3] # Shape: (batch_size, sequence_length, hidden_size)

    # Concatenate the last three hidden states
    concatenated_hidden_states = Concatenate()([
        last_hidden_state[:, 0, :],
        second_last_hidden_state[:, 0, :],
        third_last_hidden_state[:, 0, :]
    ])  # Shape: (batch_size, 3 * hidden_size)
    print(f"Shape of hidden_states: {len(hidden_states)}")
    print(f"Shape of last_hidden_state: {last_hidden_state.shape}")
    print(f"Shape of second_last_hidden_state: {second_last_hidden_state.shape}")
    print(f"Shape of third_last_hidden_state: {third_last_hidden_state.shape}")
    print(f"Shape of concatenated_hidden_states: {concatenated_hidden_states.shape}")

    bottleneck_output = Dense(bottleneck_dim, activation='relu')(concatenated_hidden_states)
    dropout_output = Dropout(0.1)(bottleneck_output)
    logits = Dense(num_labels, activation='softmax')(dropout_output)

    model = Model(inputs=[input_ids, attention_mask, token_type_ids], outputs=logits)
    return model


### Model training with simple data

In [ ]:
# Create and compile Model 1
model_1 = create_model_1(bert_model)
model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Train Model 1
model_1.fit(dataset, epochs=3)
model_1.summary()

Shape of last_hidden_state: (None, 128, 768)
Shape of cls_output: (None, 768)
Epoch 1/3


2/2 [==============================] - 46s 177ms/step - loss: 0.7869 - sparse_categorical_accuracy: 0.3333
Epoch 2/3
2/2 [==============================] - 0s 173ms/step - loss: 0.5167 - sparse_categorical_accuracy: 1.0000
Epoch 3/3
2/2 [==============================] - 0s 96ms/step - loss: 0.1743 - sparse_categorical_accuracy: 1.0000
Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                  

In [ ]:
# Create and compile Model 2
model_2 = create_model_2(bert_model)
model_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Train Model 1
model_2.fit(dataset, epochs=3)
model_2.summary()

Shape of last_hidden_state: (None, 128, 768)
Shape of cls_output: (None, 768)
Epoch 1/3


2/2 [==============================] - 40s 278ms/step - loss: 0.5643 - sparse_categorical_accuracy: 0.6667
Epoch 2/3
2/2 [==============================] - 0s 103ms/step - loss: 0.2250 - sparse_categorical_accuracy: 1.0000
Epoch 3/3
2/2 [==============================] - 0s 164ms/step - loss: 0.0484 - sparse_categorical_accuracy: 1.0000
Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                 

In [ ]:
# Create and compile Model 3
model_3 = create_model_3(bert_model)
model_3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Train Model 1
model_3.fit(dataset, epochs=3)
model_3.summary()

Shape of hidden_states: 13
Shape of last_hidden_state: (None, 128, 768)
Shape of second_last_hidden_state: (None, 128, 768)
Shape of third_last_hidden_state: (None, 128, 768)
Shape of concatenated_hidden_states: (None, 2304)
Epoch 1/3


2/2 [==============================] - 40s 242ms/step - loss: 0.8517 - sparse_categorical_accuracy: 0.3333
Epoch 2/3
2/2 [==============================] - 0s 101ms/step - loss: 0.3156 - sparse_categorical_accuracy: 1.0000
Epoch 3/3
2/2 [==============================] - 0s 95ms/step - loss: 0.1291 - sparse_categorical_accuracy: 1.0000
Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                  

In [ ]:
# !pip install datasets

### Preparing Data

In [ ]:
from datasets import load_dataset
from transformers import BertTokenizer

def prepare_dataset(dataset_name, split, tokenizer, batch_size=16, max_length=128):
    dataset = load_dataset(dataset_name, split=split)

    def tokenize_function(examples):
        return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=max_length)

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    tokenized_dataset.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

    def gen():
        for entry in tokenized_dataset:
            yield ({'input_ids': entry['input_ids'], 'attention_mask': entry['attention_mask'], 'token_type_ids': entry['token_type_ids']}, entry['label'])

    return tf.data.Dataset.from_generator(gen,
                                          output_signature=({'input_ids': tf.TensorSpec(shape=(max_length,), dtype=tf.int32),
                                                             'attention_mask': tf.TensorSpec(shape=(max_length,), dtype=tf.int32),
                                                             'token_type_ids': tf.TensorSpec(shape=(max_length,), dtype=tf.int32)},
                                                            tf.TensorSpec(shape=(), dtype=tf.int64))).batch(batch_size)
def train_model(model, dataset, epochs=3):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    model.fit(dataset, epochs=epochs)
def evaluate_model(model, dataset):
    loss, accuracy = model.evaluate(dataset)
    print(f'Loss: {loss}, Accuracy: {accuracy}')


### train Model custom data - English

In [ ]:
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")


# Create custom model 1
model_1 = create_model_1(bert_model, bottleneck_dim=256, num_labels=4)

# Prepare the dataset
train_dataset = prepare_dataset('ag_news', 'train', tokenizer, batch_size=16, max_length=128)
eval_dataset = prepare_dataset('ag_news', 'test', tokenizer, batch_size=16, max_length=128)

# Train model 1
train_model(model_1, train_dataset, epochs=2)

# Evaluate model 1
evaluate_model(model_1, eval_dataset)


GPU found
Shape of last_hidden_state: (None, 128, 768)
Shape of cls_output: (None, 768)
Epoch 1/2


7500/7500 [==============================] - 3305s 435ms/step - loss: 0.2174 - sparse_categorical_accuracy: 0.9266
Epoch 2/2
475/475 [==============================] - 84s 168ms/step - loss: 0.2161 - sparse_categorical_accuracy: 0.9351
Loss: 0.2160799503326416, Accuracy: 0.9351315498352051
